In [40]:
from PyPDF2 import PdfReader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import chromadb
from groq import Groq


In [2]:
reader = PdfReader("../datasets/budget_speech.pdf")
number_of_pages = len(reader.pages)


In [4]:
book = ''
for page in reader.pages:
    text = page.extract_text()
    book = book + text

In [14]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)
texts = text_splitter.create_documents([book])

In [16]:
chunks = [d.page_content for d in texts]

In [20]:
client = chromadb.Client()
collection = client.create_collection(name = 'collection1')


In [21]:
ids = [str(i) for i in range (len(chunks))]
collection.add(documents = chunks, ids = ids)

In [73]:
query = 'What is custom duty rate changes made in Preciuos Metals like Platinum findings.'
results = collection.query(query_texts=[query],n_results=3)
context = ' '.join(results['documents'][0])

In [74]:
# print(context)
prompt = '''Act as an AI assistant for question answering system.
All information with respect to question will be provided and question will also be provided.
You find out the exact answer of question in provided information and give answer in proper language.
If you don't get the answer from provided information , just say "No information found in given Context".
Information : {}
Question: {}'''.format(context, query)

In [38]:
# print(prompt)

In [57]:

client = Groq(
    api_key='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',
)



In [75]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="llama-3.3-70b-versatile",
)
answer = chat_completion.choices[0].message.content

In [76]:
print(answer)

No information found in given Context. The information provided does not mention any changes to the custom duty rate for precious metals like platinum findings. It only mentions changes to duty rates for various other commodities such as crust leather, handicrafts, laboratory chemicals, and articles of jewelry, but does not specifically mention platinum findings.
